In [2]:
# item 
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path, valid_data_path
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import time


train = load_pickle('../data_valid/train.pkl')
test = load_pickle('../data_valid/test.pkl')
df = pd.concat([train, test], axis=0)


In [3]:
print(train.shape,test.shape)

(420676, 36) (57411, 36)


In [4]:
def item_time_count(df):
    # ========================= 产品当天被搜索次数和当前小时被搜索次数 =========================================
    time_features = ['hour', 'day']
    item_features = ['item_id', 'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level','item_collected_level','item_pv_level']

    item_time_df = pd.DataFrame()
    item_time_df = df[['instance_id', 'day']]

    for item_feature in tqdm(item_features):
        for time_feature in time_features:
            search_group = df.groupby([item_feature, time_feature]).count().reset_index()
            tmp_df = df[[item_feature, time_feature]]
            item_day_search = pd.merge(tmp_df, search_group, on=[item_feature, time_feature], how='left').iloc[:, -1]
            item_time_df['{}_{}_search'.format(item_feature, time_feature)] = item_day_search
    
    train_feat = item_time_df[(item_time_df['day'] >= 18) & (item_time_df['day'] <= 23)].drop('day', axis=1)
    test_feat = item_time_df[item_time_df['day']==24].drop('day', axis=1)
    dump_pickle(train_feat, path=valid_data_path+'train_feature/'+'303_user_time_count.pkl')
    dump_pickle(test_feat, path=valid_data_path+'test_feature/'+'303_user_time_count.pkl')
    
    # ======================== 产品当前被搜索距离上次的时间 ================================================
    df_tmp = df[['instance_id', 'item_id',  'context_timestamp', 'day']].copy()
    df_tmp.sort_values(['item_id', 'context_timestamp'], inplace=True)

    df_tmp['item_t-1_context_timestamp'] = df_tmp.groupby('item_id')['context_timestamp'].shift(1)
    df_tmp['item_time_diff_last_query'] = df_tmp['context_timestamp'] - df_tmp['item_t-1_context_timestamp']
    # df_tmp['item_time_diff_last_query'] = np.log1p(df_tmp['context_timestamp'] - df_tmp['item_t-1_context_timestamp'])

    train_feat = df_tmp[(df_tmp['day'] >= 18) & (df_tmp['day'] <= 23)].drop('day', axis=1)   
    train_feat = train_feat[['instance_id', 'item_time_diff_last_query']]
    
    test_feat = df_tmp[df_tmp['day']==24].drop('day', axis=1)   
    test_feat = test_feat[['instance_id', 'item_time_diff_last_query']]
    
    dump_pickle(train_feat, path=valid_data_path+'train_feature/'+'303_feature_last_query.pkl')
    dump_pickle(test_feat, path=valid_data_path+'test_feature/'+'303_feature_last_query.pkl')
    
    
    
    # ========================= 当日当前搜索距离上次的时间(商品，商标) ==========================================
    final_feat = pd.DataFrame()
    final_feat = df[['instance_id', 'day']]
    cols = ['item_id', 'item_brand_id']
    for col in tqdm(cols):
        df_select = df[[col,'day','context_timestamp']]
        df_group =  df_select.groupby([col,'day'])
        group_max = df_group['context_timestamp'].transform('max')
        group_min = df_group['context_timestamp'].transform('min')
        final_feat['item_id_diff_maxtime_{}'.format(col)] = group_max - df['context_timestamp'].values
        final_feat['item_id_diff_mintime_{}'.format(col)] = df['context_timestamp'].values -group_min
        
    train_feat = final_feat[(final_feat['day'] >= 18) & (final_feat['day'] <= 23)].drop('day', axis=1)       
    test_feat = final_feat[final_feat['day']==24].drop('day', axis=1)   

    dump_pickle(train_feat, path=valid_data_path+'train_feature/'+'303_item_diff_max_min.pkl')
    dump_pickle(test_feat, path=valid_data_path+'test_feature/'+'303_item_diff_max_min.pkl')    
        
    
    
    # ================================= 当前日期前一天的cnt ===========================================
    count_features = ['item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level','item_collected_level','item_pv_level']
    final_feat = df[count_features+['instance_id', 'day']]
    for col in count_features:
        count_name = '{}_lastday_count'.format(col)
        count_all = None
        for d in range(18, 24):
            col_cnt = df[df['day'] == d - 1].groupby(by=col)['instance_id'].count().reset_index()
            col_cnt.columns = [col, count_name]
            col_cnt['day'] = d
            count_all = pd.concat([count_all, col_cnt], axis=0)
        final_feat = pd.merge(final_feat, count_all, on=[col, 'day'], how='left')
#     final_feat = final_feat.drop(count_features+['day'], axis=1)

    train_feat = final_feat[(final_feat['day'] >= 18) & (final_feat['day'] <= 23)]  
    test_feat = final_feat[final_feat['day']==24]
    train_feat = train_feat.drop(count_features+['day'], axis=1)
    test_feat = test_feat.drop(count_features+['day'], axis=1)
    
    dump_pickle(train_feat, path=valid_data_path+'train_feature/'+'303_item_last_day_count.pkl')
    dump_pickle(test_feat, path=valid_data_path+'test_feature/'+'303_item_last_day_count.pkl')

start = time.time()
item_time_count(df)
end = time.time()
print('time elapsed {}'.format(end-start))
#     print(final_feat.columns)
#     print('the shape of {} {}'.format(mode, final_feat.shape))

#     dump_pickle(final_feat, path=raw_data_path + '{}_feature/'.format(mode) + '302_item_last_day_count.pkl')

  0%|          | 0/7 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
  0%|          | 0/2 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

time elapsed 8.447350025177002
